# Data Prepration

- Extracting Reliabilty of each User tweet
- Extracting Reputation of each User 
- Extracting Sentiment of each User tweet

## Import Data

In [1]:
import pandas as pd
import openpyxl
import xlrd

from pandas import ExcelFile

In [2]:
raw_dataset = pd.read_excel("../resources/dataset/tweets.xlsx")

In [3]:
print "shape:", raw_dataset.shape
raw_dataset.head(2)

shape: (700, 17)


,id,text,created_at,retweet_count,favorite_count,is_replay_to,source,user_id,user_screen_name,user_name,user_created_at,user_description,user_followers_count,user_friends_count,favourites_count,statuses_count,user_location
1,798171734918786944,https://t.co/ccslzitMg7 best possible news yet...,2016-11-14 14:32:25,0,0,0,Twitter for Android,1659941960,Tedescop129,Pasquale Tedesco,2013-08-10 11:49:33,NaN,107,277,31,1109,NaN
2,798027051198546048,16GB is nothing 😒 #AppleSucks,2016-11-14 04:57:30,1,0,0,Twitter for iPhone,1031397014,JordanaSays,Jordana,2012-12-23 21:46:58,NaN,539,361,11363,9539,"Pittsburgh, PA"


In [4]:
new_dataset = pd.DataFrame([id for id in raw_dataset.id], columns=["id"])

## Extracting reliabilty

Using the number of likes, retweets and replays

In [5]:
def calculate_reliabilty(tweet):
    return row.retweet_count*5 + tweet.is_replay_to*3 + row.favorite_count

In [6]:
new_dataset['reliabilty'] = [calculate_reliabilty(row) for idx, row in raw_dataset.iterrows()]

In [7]:
new_dataset.head(2)

,id,reliabilty
0,798171734918786944,0
1,798027051198546048,5


## Extracting popularity

Use number of followers of the author of the tweet

In [8]:
new_dataset['popularity'] = [row.user_followers_count for idx, row in raw_dataset.iterrows()]

In [9]:
new_dataset.head(2)

,id,reliabilty,popularity
0,798171734918786944,0,107
1,798027051198546048,5,539


# Extracting Polarity

Using the Hu and Liu (2004) "opinion lexicon" 

In [10]:
negative_words = set()
with open("../resources/Hu and Liu (2004)/negative-words.txt") as f:
    for line in f.readlines():
        if line[0] != ';':
            negative_words.add(line.strip())

In [11]:
positive_words = set()
with open("../resources/Hu and Liu (2004)/positive-words.txt") as f:
    for line in f.readlines():
        if line[0] != ';':
            positive_words.add(line.strip())

In [12]:
def calculate_polarity(text):
    
    polarity_score = 0
    for word in text.split():
        if word in negative_words:
            polarity_score -= 1
        elif word in positive_words:
            polarity_score += 1
    # print text.split(), len(negative_words), len(positive_words), polarity_score
    return polarity_score

In [13]:
new_dataset['polarity'] = [calculate_polarity(row.text) for idx, row in raw_dataset.iterrows()]

In [16]:
new_dataset.head(4)

,id,reliabilty,popularity,polarity
0,798171734918786944,0,107,1
1,798027051198546048,5,539,0
2,797938783840653056,0,26,-3
3,797876478843047936,0,133,0


## Export Data

In [15]:
import openpyxl
from pandas import ExcelWriter

writer = ExcelWriter('../resources/dataset/preprocessedDataset.xlsx')
new_dataset.to_excel(writer,'Sheet1')
writer.save()